In [3]:
import json # Import Python's built-in JSON library
from datasets import Dataset, DatasetDict # Keep these
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# --- Load from SQuAD-formatted JSON MANUALLY ---
data_filepath = 'squad_format_qa_pairs.json' # Make sure this path is correct
print(f"Loading QA data from SQuAD JSON file: {data_filepath}")

try:
    # Open and load the JSON file using Python's json library
    with open(data_filepath, 'r', encoding='utf-8') as f:
        squad_data = json.load(f)
    print("Successfully loaded JSON data manually.")

    # --- Flatten the nested structure ---
    processed_examples = []
    # Access the 'data' list directly from the loaded dictionary
    for article in squad_data['data']:
         title = article['title']
         for paragraph in article['paragraphs']:
             context = paragraph['context']
             for qa in paragraph['qas']:
                 # Handle cases where 'answers' might be empty (like SQuAD v2 unanswerable)
                 # Although your example file doesn't seem to have empty answers
                 if qa['answers']:
                     first_answer = qa['answers'][0] # Take the first answer provided
                     answer_text = first_answer['text']
                     answer_start = int(first_answer['answer_start']) # Ensure it's an integer
                 else:
                     # Handle unanswerable questions if they existed
                     answer_text = ""
                     answer_start = -1 # Or some other indicator

                 processed_examples.append({
                     'id': qa['id'],
                     'title': title,
                     'context': context,
                     'question': qa['question'],
                     # Store the flattened answer info
                     'answer_text': answer_text,
                     'answer_start': answer_start
                     # We will re-create the nested 'answers' dict AFTER creating the Dataset
                 })

    # Convert the list of processed dictionaries into a Hugging Face Dataset
    if not processed_examples:
         print("ERROR: No examples were processed. Check JSON structure and processing loop.")
         raise ValueError("Processed examples list is empty.")

    processed_dataset_flat = Dataset.from_list(processed_examples)
    print("\nFlattened dataset (intermediate):")
    print(processed_dataset_flat)

    # --- Re-create the nested 'answers' column expected by preprocessing ---
    def format_answers_column(example):
        # Reconstruct the dictionary structure required by the QA processing function
        return {
            "answers": {
                "text": [example['answer_text']],
                "answer_start": [example['answer_start']]
             }
        }

    # Apply the formatting function to add the 'answers' column
    # Remove the temporary flat columns afterwards
    final_processed_dataset = processed_dataset_flat.map(
        format_answers_column,
        remove_columns=['answer_text', 'answer_start']
    )

    print("\nFinal processed dataset with 'answers' column:")
    print(final_processed_dataset)
    print("\nExample final data structure:")
    print(final_processed_dataset[0]) # Check structure: id, title, context, question, answers{...}


except FileNotFoundError:
    print(f"ERROR: Data file {data_filepath} not found.")
    raise
except json.JSONDecodeError:
    print(f"ERROR: Could not parse {data_filepath}. Check if it's valid JSON.")
    raise
except KeyError as e:
    print(f"ERROR processing JSON data: Missing expected key {e}. Check SQuAD structure.")
    raise
except Exception as e:
    print(f"An unexpected error occurred during data loading/processing: {e}")
    raise

# --- Split Data (Using the final processed dataset) ---
# This part remains the same as before, but uses 'final_processed_dataset'
total_size = len(final_processed_dataset)
test_size_fraction = 0.2 # Use 20% for evaluation
min_eval_samples = 5

if total_size <= min_eval_samples * 2:
    print(f"Warning: Dataset size ({total_size}) is very small. Using first {min(total_size, min_eval_samples)} for eval.")
    eval_indices = range(min(total_size, min_eval_samples))
    train_indices = range(min(total_size, min_eval_samples), total_size)
    if not list(train_indices): # Check if range is empty
         train_indices = eval_indices
else:
    train_indices, eval_indices = train_test_split(
        range(total_size),
        test_size=test_size_fraction,
        random_state=42
    )

train_dataset = final_processed_dataset.select(train_indices)
eval_dataset = final_processed_dataset.select(eval_indices)

# Create the final DatasetDict
dataset_dict = DatasetDict({'train': train_dataset, 'eval': eval_dataset})
print("\nCreated DatasetDict with train/eval splits:")
print(dataset_dict)
print(f"Training examples: {len(train_dataset)}")
print(f"Evaluation examples: {len(eval_dataset)}")

Loading QA data from SQuAD JSON file: squad_format_qa_pairs.json
Successfully loaded JSON data manually.

Flattened dataset (intermediate):
Dataset({
    features: ['id', 'title', 'context', 'question', 'answer_text', 'answer_start'],
    num_rows: 25
})


Map: 100%|██████████████████████████████████████████████████| 25/25 [00:00<00:00, 4590.36 examples/s]


Final processed dataset with 'answers' column:
Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 25
})

Example final data structure:
{'id': 'laz_qa_001', 'title': 'Lazada Pembatalan', 'context': 'Mengikut dasar pembatalan Lazada, anda boleh membatalkan pesanan anda terus dari akaun Lazada anda sebelum pesanan dihantar keluar dari Lazada atau gudang penjual.', 'question': 'Bagaimana jika saya ingin membatalkan produk LazMall saya?', 'answers': {'answer_start': [0], 'text': ['Mengikut dasar pembatalan Lazada, anda boleh membatalkan pesanan anda terus dari akaun Lazada anda sebelum pesanan dihantar keluar dari Lazada atau gudang penjual.']}}

Created DatasetDict with train/eval splits:
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 20
    })
    eval: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5
    })
})
Training example

ABOUT THE INPUT (CODE) & OUTPUT

Detailed Explanation & Justification: Project 3, Level 1, Step 4 (Loading & Formatting Custom SQuAD JSON)

(To be placed in a Markdown cell in your Jupyter Notebook after the successful code execution)

Overall Goal of this Code Block: To load the custom Malay e-commerce QA data (which we manually created in the standard SQuAD JSON format) into memory, transform it into the specific structure expected by Hugging Face's datasets library and QA fine-tuning tools, and finally split it into training and evaluation sets.

Code Block 1: Importing Libraries & Manual JSON Loading

import json # Import Python's built-in JSON library
from datasets import Dataset, DatasetDict # Keep these
import pandas as pd # Keep pandas for potential inspection
import numpy as np
from sklearn.model_selection import train_test_split # For splitting

# --- Load from SQuAD-formatted JSON MANUALLY ---
data_filepath = 'squad_format_qa_pairs.json' # Make sure this path is correct
print(f"Loading QA data from SQuAD JSON file: {data_filepath}")

try:
    # Open and load the JSON file using Python's json library
    with open(data_filepath, 'r', encoding='utf-8') as f:
        squad_data = json.load(f)
    print("Successfully loaded JSON data manually.")

# ... [Error handling code omitted for brevity] ...
Use code with caution.
Python
What Was Done: We imported necessary libraries and loaded the content of squad_format_qa_pairs.json into a Python dictionary called squad_data.

How It Was Done:

import json: Imported Python's standard library for working with JSON data.

from datasets import Dataset, DatasetDict: Imported classes from the Hugging Face datasets library needed later.

data_filepath = '...': Defined the path to our JSON data file.

with open(..., 'r', encoding='utf-8') as f:: This is the standard Python way to safely open a file. 'r' means read mode. encoding='utf-8' is crucial for handling text files, especially those with non-English characters (like Malay), ensuring characters are interpreted correctly. The with statement automatically closes the file even if errors occur.

squad_data = json.load(f): This function from the json library reads the text content from the file object f and parses it into a Python dictionary/list structure based on the JSON format.

Why It Was Done (Justification & Theory):

Control: Unlike the previous attempt with load_dataset('json', ...) which misinterpreted the structure, loading manually with json.load gives us full control over the raw data structure before further processing.

Standard Practice: Using with open(...) is the recommended way to handle files in Python. Specifying UTF-8 encoding prevents potential character errors with Malay text.

Corresponding Output: Successfully loaded JSON data manually. confirms this step worked.

Code Block 2: Flattening the Nested JSON Structure

# --- Flatten the nested structure ---
    processed_examples = []
    # Access the 'data' list directly from the loaded dictionary
    for article in squad_data['data']:
         title = article['title']
         for paragraph in article['paragraphs']:
             context = paragraph['context']
             for qa in paragraph['qas']:
                 # Handle cases where 'answers' might be empty (like SQuAD v2 unanswerable)
                 if qa['answers']:
                     first_answer = qa['answers'][0] # Take the first answer provided
                     answer_text = first_answer['text']
                     answer_start = int(first_answer['answer_start']) # Ensure it's an integer
                 else:
                     answer_text = ""
                     answer_start = -1 # Indicator for no answer

                 processed_examples.append({
                     'id': qa['id'],
                     'title': title,
                     'context': context,
                     'question': qa['question'],
                     # Store the flattened answer info
                     'answer_text': answer_text,
                     'answer_start': answer_start
                 })
    # ... [Error handling code omitted for brevity] ...
Use code with caution.
Python
What Was Done: We iterated through the nested structure of the loaded squad_data dictionary (articles -> paragraphs -> question-answer sets) and extracted the key information for each individual QA pair into a "flat" list of dictionaries (processed_examples).

How It Was Done:

processed_examples = []: Initialized an empty list to store the results.

for article in squad_data['data']:: Looped through each "article" dictionary inside the top-level "data" list.

title = article['title']: Extracted the title for the current article.

for paragraph in article['paragraphs']:: Looped through each "paragraph" dictionary within the current article.

context = paragraph['context']: Extracted the context text for the current paragraph.

for qa in paragraph['qas']:: Looped through each question-answer set dictionary (qa) within the current paragraph.

if qa['answers']:: Checked if the answers list associated with the current question is not empty (handling potential SQuAD v2 unanswerable format, though not present in your specific JSON).

first_answer = qa['answers'][0]: Assumed the first answer in the list is the primary one we want to use.

answer_text = first_answer['text']: Extracted the answer text string.

answer_start = int(first_answer['answer_start']): Extracted the starting character index and explicitly converted it to an integer.

processed_examples.append({...}): Created a new dictionary for each QA pair containing all the relevant extracted fields (id, title, context, question, and the flattened answer_text, answer_start). This dictionary was added to the processed_examples list.

Why It Was Done (Justification & Theory):

Data Transformation: The nested SQuAD JSON is human-readable but not ideal for direct use with many table-based tools or the Hugging Face Dataset object constructor (Dataset.from_list). Flattening creates a structure where each row represents a single training example (one QA pair).

Easier Processing: This flat list is much easier to convert into a Dataset object in the next step.

Explicit Handling: Looping explicitly ensures we process every QA pair correctly, even across different articles and paragraphs. Taking the first answer is a common simplification for training QA models when multiple annotations exist.

Code Block 3: Creating Intermediate Hugging Face Dataset

# Convert the list of processed dictionaries into a Hugging Face Dataset
    if not processed_examples:
         # ... [Error handling] ...
         raise ValueError("Processed examples list is empty.")

    processed_dataset_flat = Dataset.from_list(processed_examples)
    print("\nFlattened dataset (intermediate):")
    print(processed_dataset_flat)
Use code with caution.
Python
What Was Done: Converted the Python list of flat dictionaries (processed_examples) into a Hugging Face Dataset object.

How It Was Done:

Dataset.from_list(processed_examples): This is a constructor method from the datasets library. It takes a list of dictionaries (where each dictionary represents a row and keys are column names) and efficiently creates an Apache Arrow-backed Dataset object. Arrow provides memory efficiency and speed for data operations.

Why It Was Done (Justification & Theory):

Hugging Face Ecosystem: The datasets library is tightly integrated with transformers. Using the Dataset object allows seamless use of powerful methods like .map() for efficient tokenization and integration with the Trainer API. It's the standard data structure in this ecosystem.

Corresponding Output:

Flattened dataset (intermediate):
Dataset({
    features: ['id', 'title', 'context', 'question', 'answer_text', 'answer_start'],
    num_rows: 25
})
Use code with caution.
This output confirms the conversion worked, showing the detected features (columns) and the correct number of rows (25 QA pairs).

Code Block 4: Reconstructing Nested 'answers' Column

# --- Re-create the nested 'answers' column expected by preprocessing ---
    def format_answers_column(example):
        # Reconstruct the dictionary structure required by the QA processing function
        return {
            "answers": {
                "text": [example['answer_text']],
                "answer_start": [example['answer_start']]
             }
        }

    # Apply the formatting function to add the 'answers' column
    # Remove the temporary flat columns afterwards
    final_processed_dataset = processed_dataset_flat.map(
        format_answers_column,
        remove_columns=['answer_text', 'answer_start']
    )

    print("\nFinal processed dataset with 'answers' column:")
    print(final_processed_dataset)
    print("\nExample final data structure:")
    print(final_processed_dataset[0])
Use code with caution.
Python
What Was Done: We defined a function format_answers_column to create the specific nested dictionary structure required for the answers field ({'text': [...], 'answer_start': [...]}). We then applied this function to every row of the intermediate dataset using .map(), creating a new answers column and removing the temporary flat answer_text and answer_start columns.

How It Was Done:

def format_answers_column(example): ...: Defined a Python function that takes a single row (represented as a dictionary example) as input.

return {"answers": {...}}: The function constructs and returns the required nested dictionary structure using the values from the input example. It wraps the text and start index in lists, as expected by the SQuAD format reader in datasets/transformers.

processed_dataset_flat.map(...): The .map() method is a powerful feature of Hugging Face datasets. It efficiently applies a given function (format_answers_column) to each example (row) in the dataset.

remove_columns=['answer_text', 'answer_start']: This argument tells .map() to drop the specified columns after the new answers column has been created, keeping the dataset clean.

Why It Was Done (Justification & Theory):

Compatibility: The standard Hugging Face QA preprocessing scripts (like the complex preprocess_qa_examples function we'll use next) are written to expect the answers field in this specific nested dictionary format. We needed to transform our flattened data back into this structure.

Efficiency of .map(): Using .map() is much more efficient than iterating through a Pandas DataFrame row by row, especially for larger datasets, because it leverages the underlying Apache Arrow format and can often use parallel processing.

Corresponding Output:

Final processed dataset with 'answers' column:
Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 25
})
Example final data structure:
{'id': 'laz_qa_001', ..., 'answers': {'answer_start': [0], 'text': ['Mengikut...']}}
Use code with caution.
This confirms the .map() worked, the final dataset has the correct 5 features, and the example shows the answers column now has the required nested dictionary structure.

Code Block 5: Splitting Data into Train/Eval Sets

# --- Split Data (Using the final processed dataset) ---
total_size = len(final_processed_dataset)
test_size_fraction = 0.2 # Use 20% for evaluation
min_eval_samples = 5

if total_size <= min_eval_samples * 2:
    # ... [Handling very small datasets] ...
else:
    train_indices, eval_indices = train_test_split(
        range(total_size),
        test_size=test_size_fraction,
        random_state=42
    )

train_dataset = final_processed_dataset.select(train_indices)
eval_dataset = final_processed_dataset.select(eval_indices)

# Create the final DatasetDict
dataset_dict = DatasetDict({'train': train_dataset, 'eval': eval_dataset})
print("\nCreated DatasetDict with train/eval splits:")
# ... [Rest of print statements] ...
Use code with caution.
Python
What Was Done: The final processed dataset was split into two separate datasets: one for training (train_dataset) and one for evaluation (eval_dataset), which were then stored together in a DatasetDict.

How It Was Done:

total_size = len(final_processed_dataset): Got the total number of examples (25).

Defined test_size_fraction (0.2 or 20%) and min_eval_samples (5).

Included logic to handle very small datasets (not strictly needed here with 25 samples but good practice).

train_test_split(range(total_size), ...): Used scikit-learn's train_test_split function. Instead of splitting the data directly, we split the indices (from 0 to 24). This is often efficient. test_size=0.2 asked for 20% of indices for the evaluation set. random_state=42 ensures the split is the same every time the code is run (reproducibility). It returned two lists: train_indices and eval_indices.

final_processed_dataset.select(indices): Used the .select() method of the Hugging Face Dataset to create new datasets containing only the examples at the specified indices.

dataset_dict = DatasetDict({'train': ..., 'eval': ...}): Created a DatasetDict, which is a dictionary-like object from the datasets library specifically designed to hold multiple dataset splits (like 'train', 'validation', 'test') under convenient keys.

Why It Was Done (Justification & Theory):

Model Evaluation: It is absolutely critical in machine learning to evaluate your model's performance on data it has never seen during training. Training and evaluating on the same data gives an overly optimistic view and doesn't reflect how the model will perform on new, real-world examples (overfitting).

Standard Practice: Splitting data into dedicated training and evaluation (or validation/test) sets is a fundamental step in the ML workflow.

DatasetDict: Using DatasetDict is the standard way to organize multiple splits within the Hugging Face ecosystem, making it easy to pass them to the Trainer.

Corresponding Output:

Created DatasetDict with train/eval splits:
DatasetDict({
    train: Dataset({ ... num_rows: 20 })
    eval: Dataset({ ... num_rows: 5 })
})
Training examples: 20
Evaluation examples: 5
Use code with caution.
This confirms the split resulted in 20 training examples and 5 evaluation examples, organized correctly within the dataset_dict.

In [5]:
#Sub-Step 5.2: Load the Tokenizer

from transformers import AutoTokenizer

# Define the model checkpoint again (must match the model loaded later)
model_checkpoint = "bert-base-multilingual-cased"
print(f"\nLoading tokenizer for: {model_checkpoint}")

try:
    # AutoTokenizer automatically fetches the correct tokenizer class and vocabulary
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    print("Tokenizer loaded successfully.")
    print(f"Tokenizer type: {type(tokenizer)}")

    # Optional: Test the tokenizer on a sample Malay text
    sample_text = "Ini adalah contoh ayat."
    encoded = tokenizer(sample_text)
    print(f"\nEncoding for '{sample_text}':")
    print(encoded)
    print("Decoded back:", tokenizer.decode(encoded['input_ids']))

except Exception as e:
    print(f"ERROR: Failed to load the tokenizer. Check model checkpoint name and internet connection. Error: {e}")
    raise


Loading tokenizer for: bert-base-multilingual-cased
Tokenizer loaded successfully.
Tokenizer type: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>

Encoding for 'Ini adalah contoh ayat.':
{'input_ids': [101, 25336, 10784, 65012, 71072, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
Decoded back: [CLS] Ini adalah contoh ayat. [SEP]


In [6]:
#Sub-Step 5.3: Define the Preprocessing Function
# Define constants for processing
# Max total length for Question + Context after tokenization
max_length = 384
# How much overlap between chunks when context is split
doc_stride = 128

print(f"Using max_length={max_length}, doc_stride={doc_stride}")

# This function processes ONE BATCH of examples
def preprocess_qa_examples_for_training(examples):
    # Pad questions on the right, contexts on the right
    # Extract questions and contexts from the batch
    # Ensure questions are lists of strings, contexts are lists of strings
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]
    answers = examples["answers"] # List of dictionaries {'text':[], 'answer_start':[]}

    # Tokenize question and context pairs
    # return_overflowing_tokens=True creates multiple samples (features) if context is too long
    # return_offsets_mapping=True gives character spans for each token (needed for answer mapping)
    tokenized_examples = tokenizer(
        questions,
        contexts,
        truncation="only_second",  # Truncate context only if needed (prefer keeping question)
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",      # Pad all sequences to max_length
    )

    # The tokenizer returns a list of features mapped to original examples
    # `overflow_to_sample_mapping` tells us which original example each feature came from.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # `offset_mapping` gives char start/end for each token in each feature. Pop it too.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Now, we need to compute start_positions and end_positions for each feature
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # Iterate through each generated feature (tokenized sequence)
    for i, offsets in enumerate(offset_mapping):
        # Get the token IDs and find the CLS token index for this feature
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Get the sequence IDs (0 for question, 1 for context, None for special tokens)
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Identify the corresponding original example index
        sample_index = sample_mapping[i]
        # Get the answers associated with that original example
        current_answers = answers[sample_index]

        # If no answers are given (e.g., for SQuAD v2 unanswerable), label is (CLS, CLS)
        if len(current_answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Get the character start and end of the answer in the original context
            start_char = current_answers["answer_start"][0]
            end_char = start_char + len(current_answers["text"][0])

            # Find the token indices that correspond to the start and end of the context within the feature
            token_start_index = 0
            while sequence_ids[token_start_index] != 1: # sequence_ids value '1' marks the context
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Check if the answer span is completely outside the current feature's context span
            # If it is, the label is (CLS, CLS)
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise, the answer is within this feature's context span.
                # Find the token index corresponding to the answer's start character.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                # Find the token index corresponding to the answer's end character.
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1

                # Assign the token indices. Add token_start_index-1 because we went one step too far.
                # Add token_end_index+1 because the end index is inclusive.
                tokenized_examples["start_positions"].append(token_start_index - 1)
                tokenized_examples["end_positions"].append(token_end_index + 1) # Check if +1 needed depends on tokenizer/model head

    # Return the dictionary containing input_ids, attention_mask, start_positions, end_positions
    return tokenized_examples

print("\nPreprocessing function defined.")

Using max_length=384, doc_stride=128

Preprocessing function defined.


In [10]:
# --- This is Python Code Cell ---

print("\nApplying preprocessing and tokenization to datasets...")

# Ensure dataset_dict is available from the previous step
if not 'dataset_dict' in locals() or not isinstance(dataset_dict, DatasetDict):
     print("ERROR: 'dataset_dict' not found or not a DatasetDict. Run previous steps.")
     raise NameError("'dataset_dict' not properly defined.")

try:
    # Apply the function using map.
    # batched=True processes multiple rows simultaneously for speed.
    # remove_columns removes the original text columns, leaving only model inputs/outputs.
    tokenized_datasets = dataset_dict.map(
        preprocess_qa_examples_for_training, # The function defined earlier
        batched=True,
        remove_columns=dataset_dict["train"].column_names # Remove original cols like 'id', 'context' etc.
    )
    print("\nPreprocessing complete.")
    print("\nTokenized dataset structure:")
    print(tokenized_datasets)
    print("\nExample tokenized train features keys:")
    # Check the first example's keys
    print(tokenized_datasets['train'][0].keys())

except Exception as e:
    print(f"ERROR occurred during .map() operation: {e}")
    # Common issues: Errors inside the preprocessing function, issues with batching, missing columns.
    raise

# --- End of Python Code Cell ---


Applying preprocessing and tokenization to datasets...


Map: 100%|█████████████████████████████████████████████████████| 5/5 [00:00<00:00, 646.41 examples/s]


Preprocessing complete.

Tokenized dataset structure:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 20
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5
    })
})

Example tokenized train features keys:
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


**Action - Analyze Output:**

*   Run the code cell above.
*   Monitor the progress bar (it might be quick for your small dataset).
*   Verify the `Preprocessing complete` message appears.
*   Look at the structure of `tokenized_datasets`. It should still be a `DatasetDict` with `train` and `eval` splits.
*   Check the `features` listed for the datasets. They should now include `input_ids`, `attention_mask`, `start_positions`, and `end_positions`, and the original text columns should be gone.
*   Check the example keys printed – they should match the new features.

**Goal:** Have the `tokenized_datasets` variable holding the fully preprocessed, numerical data ready to be fed into the `Trainer` in the next step.

In [11]:
#Sub-Step 6.2: Write and Run Code to Load Model

from transformers import AutoModelForQuestionAnswering
import torch # Ensure torch is imported if checking for CUDA

# Re-define model_checkpoint if in a new cell/session, must match tokenizer
model_checkpoint = "bert-base-multilingual-cased"
print(f"\nLoading pre-trained model with QA head: {model_checkpoint}")

try:
    # AutoModelForQuestionAnswering automatically finds the correct class (BertForQuestionAnswering)
    # and loads the pre-trained weights for the specified checkpoint.
    # It adds a QA head (usually two linear layers) on top of the base BERT model.
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

    # Optional: Check which device the model loaded onto (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device) # Ensure model is on the correct device (especially important if using GPU later)
    print(f"Model loaded successfully and moved to device: {model.device}")

except Exception as e:
    print(f"ERROR: Failed to load the model '{model_checkpoint}'.")
    print(f"Check model checkpoint name, internet connection, and available memory.")
    print(f"Error details: {e}")
    raise


Loading pre-trained model with QA head: bert-base-multilingual-cased


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully and moved to device: cpu
